In [1]:
import os
import numpy as np
import shutil
from tqdm import tqdm
copyfile = shutil.copyfile
exists = os.path.exists

In [2]:
cd ~/data/fMRI_InData/

/mmfs1/data/aglinska/data/fMRI_InData


In [3]:
folders = [folders for folders in os.listdir('abide_bids') if folders.startswith('sub-')]
folders.sort()
folders = np.array(folders)
len(folders)

1109

In [4]:
has_t1w = np.array([os.path.exists(os.path.join(f'./abide_bids/{sub}/anat/{sub}_T1w.nii')) for sub in folders])
has_t1wo = np.array([os.path.exists(os.path.join(f'./abide_bids/{sub}/anat/{sub}_T1wo.nii')) for sub in folders])
folders = folders[has_t1wo]

In [5]:
subs = folders

In [6]:
if not os.path.exists('abide_bids_val'):
    os.mkdir('abide_bids_val')
    
    
src = './abide_bids/dataset_description.json'
dest ='./abide_bids_val/dataset_description.json' 
copyfile(src,dest)

'./abide_bids_val/dataset_description.json'

In [8]:
for sub in tqdm(subs):
    if not exists(f'./abide_bids_val/{sub}'):
        os.mkdir(f'./abide_bids_val/{sub}')
        os.mkdir(f'./abide_bids_val/{sub}/anat')

        src = f'./abide_bids/{sub}/anat/{sub}_T1wo.nii'
        dest = f'./abide_bids_val/{sub}/anat/{sub}_T1w.nii'
        copyfile(src,dest)

100%|██████████| 1099/1099 [07:44<00:00,  2.36it/s]


#!/bin/bash

#SBATCH --job-name=job_mriqc_abide
#SBATCH --output=./qc_jobs/output_qc_job_%a
#SBATCH --error=./qc_jobs/error_qc_job_%a

#SBATCH --ntasks=1
#SBATCH --time=1:00:00
#SBATCH --mem=32gb
#SBATCH --array=100-1099:200

#rm -rf job_mriqc_abide.slurm;nano job_mriqc_abide.slurm;sbatch job_mriqc_abide.slurm
#$SLURM_ARRAY_TASK_ID


#rm -rf output_fmriprep_ABIDEII-NYU_2_%a
#rm -rf error_fmriprep_ABIDEII-NYU_2_%a

cd data/fMRI_InData/abide_bids_val/
arr=($(ls -d sub*))
echo ${#arr[@]}
echo ${arr[$SLURM_ARRAY_TASK_ID]}

cd 

module load singularity/

SINGULARITY_LOCALCACHEDIR=/scratch/aglinska/
SINGULARITY_TMPDIR=/scratch/aglinska/
SINGULARITY_CACHEDIR=/scratch/aglinska/

export SINGULARITY_LOCALCACHEDIR
export SINGULARITY_TMPDIR
export SINGULARITY_CACHEDIR


bids_root=~/data/fMRI_InData/abide_bids_val/
output_folder=~/qc_reports/
sing_image=mriqc21
singularity run --bind /data/aglinska/scratch:/scratch --cleanenv $sing_image $bids_root $output_folder -m T1w participant --participant-label ${arr[$SLURM_ARRAY_TASK_ID]}
#singularity run --bind /data/aglinska/scratch:/scratch --cleanenv $sing_image $data_dir $output_dir participant --participant-label $sub --fs-no-reconall --task-id 'rest' --fs-license-file $fs_lic --ignore slicetiming